# Test 06: TensorFlow/Keras Suppression

**Purpose:** Verify that transformers correctly ignores TensorFlow and Keras, preventing import errors and warnings.

**Background:**
- CoCalc has Keras 3 installed, which is incompatible with transformers' TF backend
- Without suppression, `from transformers import ...` can trigger TF/Keras import errors
- The auto_update script sets environment variables to prevent this

**Expected Results:**
- `TRANSFORMERS_NO_TF=1` should be set
- `USE_TF=NO` should be set  
- `TF_CPP_MIN_LOG_LEVEL=3` should be set
- Importing transformers should NOT trigger any TF/Keras errors

**Run this on:** Both CoCalc base and Compute Server

In [ ]:
# DS776 Environment Setup & Package Update
# Configures storage paths for proper cleanup/sync, then updates introdl if needed
# If this cell fails, see Lessons/Course_Tools/AUTO_UPDATE_SYSTEM.md for help
%run ../../Lessons/Course_Tools/auto_update_introdl.py

In [ ]:
# Check TF/Keras suppression environment variables
import os

print("=" * 60)
print("TF/KERAS SUPPRESSION ENVIRONMENT VARIABLES")
print("=" * 60)

suppress_vars = {
    'TRANSFORMERS_NO_TF': '1',
    'USE_TF': 'NO',
    'TF_CPP_MIN_LOG_LEVEL': '3',
}

all_good = True
for var, expected in suppress_vars.items():
    value = os.environ.get(var, 'NOT SET')
    if value == expected:
        status = "OK"
    elif value == 'NOT SET':
        status = "WARNING: Not set"
        all_good = False
    else:
        status = f"WARNING: Expected '{expected}', got '{value}'"
        all_good = False
    print(f"{var}: {value} [{status}]")

print()
if all_good:
    print("RESULT: All suppression variables configured correctly!")
else:
    print("RESULT: Some suppression variables not set correctly")

In [ ]:
# Test: Import transformers without TF/Keras errors
print("=" * 60)
print("TEST: Import transformers (should not trigger TF/Keras errors)")
print("=" * 60)

import warnings
import sys
from io import StringIO

# Capture any warnings during import
captured_warnings = []
def warning_handler(message, category, filename, lineno, file=None, line=None):
    captured_warnings.append(str(message))

old_showwarning = warnings.showwarning
warnings.showwarning = warning_handler

# Capture stderr for any TF messages
old_stderr = sys.stderr
sys.stderr = StringIO()

try:
    # This import would fail or produce warnings without suppression
    from transformers import AutoModel, AutoTokenizer, pipeline
    import_success = True
    import_error = None
except Exception as e:
    import_success = False
    import_error = str(e)
finally:
    stderr_output = sys.stderr.getvalue()
    sys.stderr = old_stderr
    warnings.showwarning = old_showwarning

# Report results
if import_success:
    print("PASS: transformers imported successfully")
    print(f"  - AutoModel: {AutoModel}")
    print(f"  - AutoTokenizer: {AutoTokenizer}")
    print(f"  - pipeline: {pipeline}")
else:
    print(f"FAIL: transformers import failed")
    print(f"  Error: {import_error}")

# Check for TF/Keras related warnings
tf_warnings = [w for w in captured_warnings if 'tensorflow' in w.lower() or 'keras' in w.lower() or 'tf' in w.lower()]
if tf_warnings:
    print(f"\nWARNING: {len(tf_warnings)} TF/Keras related warnings captured:")
    for w in tf_warnings[:5]:  # Show first 5
        print(f"  - {w[:100]}..." if len(w) > 100 else f"  - {w}")
else:
    print("\nPASS: No TF/Keras warnings during import")

# Check stderr for TF messages
if stderr_output.strip():
    tf_stderr = [line for line in stderr_output.split('\n') if line.strip()]
    if tf_stderr:
        print(f"\nStderr output ({len(tf_stderr)} lines):")
        for line in tf_stderr[:5]:
            print(f"  {line[:100]}")
else:
    print("PASS: No stderr output during import")

In [ ]:
# Test: Verify transformers is NOT using TensorFlow backend
print("=" * 60)
print("TEST: Verify transformers backend configuration")
print("=" * 60)

import transformers

# Check if TF is being used
try:
    from transformers.utils import is_tf_available, is_torch_available
    
    torch_available = is_torch_available()
    tf_available = is_tf_available()
    
    print(f"PyTorch available: {torch_available}")
    print(f"TensorFlow available: {tf_available}")
    
    if torch_available and not tf_available:
        print("\nPASS: transformers is using PyTorch only (TF disabled)")
    elif torch_available and tf_available:
        print("\nWARNING: Both PyTorch and TF are available")
        print("  This may cause issues but imports should still work")
    elif not torch_available:
        print("\nWARNING: PyTorch not available!")
except ImportError as e:
    print(f"Could not check backend availability: {e}")

In [ ]:
# Test: Load a model to verify everything works end-to-end
print("=" * 60)
print("TEST: Load a small model (functional test)")
print("=" * 60)

try:
    from transformers import AutoTokenizer
    
    # Use a tiny model for quick testing
    model_name = "prajjwal1/bert-tiny"
    print(f"Loading tokenizer for: {model_name}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Quick tokenization test
    test_text = "Hello, this is a test."
    tokens = tokenizer(test_text, return_tensors="pt")
    
    print(f"\nPASS: Tokenizer loaded and working")
    print(f"  Input: '{test_text}'")
    print(f"  Token IDs: {tokens['input_ids'].tolist()[0][:10]}...")
    print(f"  Attention mask shape: {tokens['attention_mask'].shape}")
    
except Exception as e:
    print(f"FAIL: Could not load model")
    print(f"  Error: {e}")

## Summary

If all tests passed:
- TF/Keras suppression environment variables are set correctly
- transformers imports without triggering TF/Keras errors
- transformers is configured to use PyTorch backend
- Models can be loaded and used normally

**Why this matters:**
- CoCalc has Keras 3.x installed which is incompatible with transformers' TF support
- Without suppression, students would see confusing TF/Keras import errors
- The auto_update script sets these variables early to prevent issues